In [0]:
#Code originally ran on Google Colab using GPU provided by Google

#Importing packages
import statistics
import copy 
import numpy as np
import pandas as pd
import tensorflow as tf
#import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler


In [0]:
#Setting random seed
RANDOM_SEED = 2020
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
#Connecting to Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Check GPU
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [0]:
#Importing data from Google Drive into pandas dataframe while parsing dates as index
df = pd.read_csv("/content/gdrive/My Drive/Ethereum.csv", parse_dates = ["date"], index_col = "date") #Change dataset name accordingly
df = df.sort_index()

In [0]:
#removing unnecesarry index column left behind by R
df = df.drop(["Unnamed: 0"], axis=1)

In [0]:
#Creating a function that will return the large or big dataset
def featureSelector(crypto, size):
  articles = "articles" + crypto
  sentiment = "sentiment" + crypto
  trend = "trend" + crypto
  wiki = "wiki" + crypto
  if size not in [0, 1]:
    print("wrong size specification")
    pass

  elif size == 0: #small dataset
    return df.loc[:,["open", "high", "low", "close", "volume", "marketCap", articles, sentiment, trend, wiki, #change articles, sentiment, trend and wiki accordingly
           "wti", "brent", "EPU", "vixClose", "target"]]
  else: #big dataset
    return df.loc[:,["open", "high", "low", "close", "volume", "marketCap", articles, sentiment, "articlesBlockchain", "sentimentBlockchain", "articlesCryptocurrency", "sentimentCryptocurrency", #change articles, sentiment accordingly
           "articlesCryptocurrencies", "sentimentCryptocurrencies", trend, "trendBlockchain", "trendCryptocurrency", "trendCryptocurrencies", wiki, "wikiBlockchain", "wikiCryptocurrency", #change trend and wiki accordingly
           "wti", "brent", "EPU", "vixClose", "target"]]

# datasetConstructor("Bitcoin"/"Ethereum"/"Ripple", 0/1) where 0 is small dataset and 1 large
# in case of error: reread from csv 
df = featureSelector("Ethereum", 1)



In [20]:
df.head()

,open,high,low,close,volume,marketCap,articlesEthereum,sentimentEthereum,articlesBlockchain,sentimentBlockchain,articlesCryptocurrency,sentimentCryptocurrency,articlesCryptocurrencies,sentimentCryptocurrencies,trendEthereum,trendBlockchain,trendCryptocurrency,trendCryptocurrencies,wikiEthereum,wikiBlockchain,wikiCryptocurrency,wti,brent,EPU,vixClose,target
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,7.98,8.47,7.98,8.17,14731700.0,715049208,0,0.000000,10,-1.380042,0,0.000000,0,0.0,1,7,1,2,1852,2006,2389,52.36,55.05,2351,12.85,1
2017-01-02,8.17,8.44,8.05,8.38,14579600.0,733331654,4,-2.764493,38,0.837092,0,0.000000,0,0.0,2,10,1,2,3338,3512,4827,52.36,55.05,24204,12.85,1
2017-01-03,8.37,10.00,8.32,9.73,33625200.0,851512055,0,0.000000,120,1.812371,24,-0.828760,0,0.0,3,15,1,3,3981,4672,5687,52.36,55.05,8985,12.85,1
2017-01-04,9.71,11.28,9.56,11.25,41051200.0,985515893,0,0.000000,181,2.519655,54,2.376397,0,0.0,3,15,1,2,4484,6338,5507,53.26,54.57,10199,11.85,0
2017-01-05,11.29,11.89,9.40,10.25,41557400.0,898497892,78,-2.816484,170,2.789309,30,2.119208,0,0.0,3,14,1,3,4615,5676,5916,53.77,54.99,13447,11.67,0


In [21]:
#Constructing train and test set. Training set size hardcoded to be 90%
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]

print(train.shape, test.shape)

(1094, 26) (122, 26)


In [22]:
#retrieving array names from dataset and removing target label from this array
features = df.columns.drop("target")
#Fit scaler on train set so that model has no information on test data.
scaler = MinMaxScaler()
scaler = scaler.fit(train[features].to_numpy()) #fit data on train data

train.loc[:, features] = scaler.transform(train[features].to_numpy()) #Scaling all features used and replacing in dataframe 
test.loc[:, features] = scaler.transform(test[features].to_numpy()) #Scaling all features used (scaler fit on train data) and replacing in dataframe


#Warning is thrown because data in train and test set are overwritten by scaled data. both train and test data reference to parts of the 'original' pandas df. 


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
# Making function that will format input data to the right dimension
def datasetCreator(X, y):
  Xs, ys = [], []
  for i in range(len(X)):
    Xs.append(X.iloc[i:i+1].to_numpy())
    ys.append(y.iloc[i])
  return np.array(Xs), np.array(ys).squeeze()

In [0]:
# Constructing the actual dataset by using the datasetCreator function
# Hardcoding target column name as it is always the same
X_train, y_train = datasetCreator(train.loc[:, features], train.loc[:, "target"])
#X_val, y_val = datasetCreator(val.loc[:, features], val.loc[:, target]) 
X_test, y_test = datasetCreator(test.loc[:, features], test.loc[:, "target"])

In [25]:
#Making sure the dataframes have equal rows and have the expected dimension
#Small dataset = 14 columns, Big dataset = 25
print(X_train.shape, y_train.shape)

(1094, 1, 25) (1094,)


In [26]:
def makeModel(X_train, y_train, config, config_accuracy):
  #reading the parameters used in the model
  n_units, n_epochs, n_batch, activation, dropout = config 

  model = keras.Sequential()
  model.add(keras.layers.LSTM(units = n_units, input_shape=(X_train.shape[1], X_train.shape[2])
                             #, return_sequences = True     #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
                              )) 
  model.add(keras.layers.Dropout(dropout))
  #model.add(keras.layers.LSTM(units = 32))    #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
  #model.add(keras.layers.Dropout(dropout))    #UNCOMMENT IN ORDER TO TEST 2ND LSTM LAYER!!
  model.add(keras.layers.Dense(units = 1, activation = activation))
  model.compile(loss= "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])
  
  #fit model
  history = model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_batch, validation_split = 0.10, shuffle = False, verbose = 0)

  #storing the accuracy of the last epoch on both train and validation data
  config_accuracy[0].append(history.history["accuracy"][-1])
  config_accuracy[1].append(history.history["val_accuracy"][-1])
  
def model_configs():
	#defining the parameters used
  n_units = [320, 384]
  n_epochs = [100, 150]
  n_batch = [32]
  activation = ["sigmoid", "tanh"]
  dropout = [0, 0.10, 0.15, 0.20]
	# create configs
  configs = list()
  for i in n_units:
    for j in n_epochs:
      for k in n_batch:
        for l in activation:
          for m in dropout:
              cfg = [i, j, k, l, m]
              configs.append(cfg)
  print("Total configs: {}".format(len(configs)))
  return configs

configList = model_configs()
for config in configList:
  config_accuracy = [[],[]]
  for i in range(5):
    makeModel(X_train, y_train, config, config_accuracy)
  n_units, n_epochs, n_batch, activation, dropout  = config
  train_acc = statistics.mean(config_accuracy[0])
  val_acc = statistics.mean(config_accuracy[1])
  print("units: {}, epoch: {}, batch: {}, activation: {}, dropout: {}, ACC: {}, VAL_ACC: {}, ACC high: {}, ACC low: {}, VAL ACC high: {}, VAL ACC low: {}".format(n_units, n_epochs, n_batch, activation, dropout, train_acc, val_acc, max(config_accuracy[0]), min(config_accuracy[0]),  max(config_accuracy[1]), min(config_accuracy[1])))


Total configs: 32
units: 320, epoch: 100, batch: 32, activation: sigmoid, dropout: 0, ACC: 0.5908536314964294, VAL_ACC: 0.5472727417945862, ACC high: 0.5924796462059021, ACC low: 0.5894308686256409, VAL ACC high: 0.5545454621315002, VAL ACC low: 0.5363636612892151
units: 320, epoch: 100, batch: 32, activation: sigmoid, dropout: 0.1, ACC: 0.5867885947227478, VAL_ACC: 0.5309090852737427, ACC high: 0.5904471278190613, ACC low: 0.5843495726585388, VAL ACC high: 0.5454545617103577, VAL ACC low: 0.5181818008422852
units: 320, epoch: 100, batch: 32, activation: sigmoid, dropout: 0.15, ACC: 0.5770325064659119, VAL_ACC: 0.5199999809265137, ACC high: 0.5873983502388, ACC low: 0.5701219439506531, VAL ACC high: 0.5272727012634277, VAL ACC low: 0.5090909004211426
units: 320, epoch: 100, batch: 32, activation: sigmoid, dropout: 0.2, ACC: 0.5762194991111755, VAL_ACC: 0.5181818008422852, ACC high: 0.5833333134651184, ACC low: 0.5691056847572327, VAL ACC high: 0.5181818008422852, VAL ACC low: 0.5181818

In [0]:
#print(config_accuracy)

[[0.4359756112098694, 0.5813007950782776, 0.5924796462059021, 0.5945122241973877, 0.5934959053993225], [0.5909090638160706, 0.48181816935539246, 0.5, 0.4909090995788574, 0.4909090995788574]]


In [0]:
#Possible baseline model that uses todays observation as tomorrows

success = 0
failure = 0
for i in range(len(y_train)):
  print
  if i == len(y_train) - 1:
    print("success: {}".format(success))
    print("failure: {}".format(failure))
    break
  elif y_train[i] == y_train[i+1]:
    success += 1
  else:
    failure += 1
  

success: 491
failure: 602


In [0]:
print(y_train[1])

1


In [0]:
print(y_train)

[0 1 1 0 1 1 0 0 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0
 1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1
 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0
 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1
 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0
 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1
 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 1 0 1
 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0
 0 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 1 1 1 1 0 0 0 0 